<a href="https://colab.research.google.com/github/ozachun/sample.py/blob/main/sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from datetime import datetime, timedelta
import pandas as pd 
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#サンプルデータをダウンロード
data= pd.read_csv('sample_data/is_valid.csv')
print(data)

In [ ]:
data = data.set_axis(['is_available', 'created_at'], axis=1)#列の名前を変更
data01 = data.dropna(axis = 'rows')#欠損値の削除
data01['created_at'] = pd.to_datetime(data01['created_at'])#型の変更
df = data01[(data01['created_at'] >= dt.datetime(2018,3,1)) & (data01['created_at'] < dt.datetime(2021,3,15))]
print(df) #2018/3/1(水)～2021/3/14(土)の期間の応募

In [ ]:
df["created_at"] = pd.to_datetime(df["created_at"])#型の変更
df.set_index('created_at', inplace=True)#これでインデックスがDatetimeIndexとなる。
print(df)

In [ ]:
df_daily = df.resample('1D').sum()#2018/3/1から2021/3/14までの１日ごとの応募数にまとめる
print(df_daily)

In [ ]:
df_daily01 = df_daily.reset_index()#列の計算をするためにindexをリセット
print(df_daily01)

In [ ]:
df5 = pd.concat([df_daily01["created_at"]] * 24, 1).stack().reset_index(drop=True)#応募日の列を24行分コピー
df10 = pd.concat([df_daily01["is_available"]] * 24, 1).stack().reset_index(drop=True)#応募数の列を24行分コピー
df50 = pd.concat([df5, df10], axis=1)#上記24倍した列を一つの表にまとめる
print(df50)

In [ ]:
df_hour = df.resample('1H').sum()#2018/3/1から2021/3/14までの１時間ごとの応募数
df_hour = df_hour.reset_index()#列の計算をするためにindexをリセット
print(df_hour)

In [ ]:
df50 = df50.assign(ID=range(len(df50)))#ID列を追加
df_hour = df_hour.assign(ID=range(len(df_hour)))#ID列を追加
print(df50)
print(df_hour)

In [ ]:
df100 = pd.merge(df50, df_hour)#表の結合（5列）
print(df100)

In [ ]:
#1時間ごとの応募割合の計算＋表への結合(6列)
percent = df100["is_available"]/df100[1]
df100 = pd.concat([df100, percent], axis=1)
print(df100)

In [ ]:
df100 = df100.set_axis(['a', 'b', 'ID', 'c', 'd', 'e'], axis=1)#列名を英語に変更
print(df100)

In [ ]:
#累積応募割合を計算＋列に追加(7列)
df_f = df100["e"].cumsum(axis=0)
df_f = pd.concat([df_f, df_f], axis=1)
df_f = df_f.set_axis(['f', 'g'], axis=1)
df100 = pd.concat([df100, df_f["f"]], axis=1)
print(df100)

In [ ]:
print(df100) #1時は24n(n=0,1,...,1109),２時は24n+1(n=0,1,...,1109),・・・

In [ ]:
#1日の応募割合を計算する①累積和-経過日数をしたい
results = []
for i in range(len(df100)):
  x = i//24 #商を計算
  results.append(x)
print(results)

In [ ]:
#上記で計算した列をdf100に追加（7列目）
results = pd.DataFrame(results)
df100 = pd.concat([df100, results], axis=1)
df100 = df100.set_axis(['a', 'b', 'ID', 'c', 'd', 'e', 'f', 'g'], axis=1)#列名の名前の変更
print(df100)

In [ ]:
#引き算の結果を列に追加(8列目)
df100["h"] = df100["f"] - df100["g"]
print(df100)

In [18]:
#日付から曜日を判定し、表に追加 0:月曜日,...,6:日曜日
import datetime as dt

dates = []
for i in range(0,len(df100)):
  datey = df100.iloc[i,0]
  datey_y = datey.year
  datey_m = datey.month
  datey_d = datey.day
  datey = dt.date(datey_y, datey_m, datey_d).weekday()
  dates.append(datey)

In [ ]:
print(dates[2])
print(len(dates))
print(dates)

In [20]:
#曜日の値をdfに追加(10列)
df100 = df100.assign(date=dates)

In [ ]:
print(df100.head(50))

In [22]:
#１時間ごとに行を抽出
tables = []
for i in range(0,24):
  table = []
  table = df100[i::24]
  tables.append(table)

In [ ]:
tables_hour = tables[0]
print(tables_hour)
print(tables_hour["date"])
print(tables_hour[tables_hour['date'].isin([1])])

In [ ]:
#曜日ごとに抽出
tables_hourdate = []
tables_hourdates =[]
for i in range(0,24):
  tables_hour = []
  tables_hour = tables[i] #0時時点の抽出
  print(tables_hour)
  for j in range(0,7):
    print(tables_hour[tables_hour['date'].isin([j])]) #0時かつ月曜日を抽出
    tables_hourdate = np.array(tables_hour[tables_hour['date'].isin([j])])
    print(tables_hourdate)
    tables_hourdates = np.append(tables_hourdate, tables_hourdate)

In [ ]:
print(tables_hourdates.shape)

In [ ]:
print(tables[0])#0～1時時点の抽出結果
print(tables[23])#23～24時時点の抽出結果

In [27]:
#2018/3/1の300日以降のデータを抽出
ts = []
for i in range(0,24):
  t = tables[i][300:]
  ts.append(t)

In [ ]:
print(ts[0])#0～1時時点かつ2018/3/1の300日以降の抽出結果
print(ts[23])#23～24時時点かつ2018/3/1の300日以降の抽出結果

In [ ]:
x = ts[9]['a']
y = ts[9]['h']
plt.rcParams["figure.figsize"] = (40, 8)
plt.plot(x, y)

In [ ]:
#2021/3/14までの165日のデータの平均を抽出
ts_mean = []
for i in range(0,24):
  t_mean = ts[i].tail(165).mean()
  ts_mean.append(t_mean)

In [ ]:
print(ts_mean[0])#0～1時時点の平均を抽出
print(ts_mean[23])#23～24時時点の平均を抽出

In [32]:
a_s =[]
for i in range(0,24):
  a = ts_mean[i][6:7]
  a_s.append(a)

In [ ]:
print(a_s[0])#１時時点での平均応募割合
print(a_s[9])#10時時点での平均応募割合